# Content-Based Filtering System - Data Generation

This notebook generates synthetic data for our Content-Based Filtering recommendation system:
- 10,000+ products with rich features
- 100 customers with diverse preferences
- Realistic interaction patterns

In [0]:
%pip install faker
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog", "retail_consumer_goods")
dbutils.widgets.text("schema", "conversational_commerce_search")
dbutils.widgets.text("image_schema", "images")
dbutils.widgets.text("volume_name", "product_images")
dbutils.widgets.dropdown("Generate Product Images", choices=["false"], defaultValue="false", label="Generate Product Images")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
image_schema = dbutils.widgets.get("image_schema")
volume_name = dbutils.widgets.get("volume_name")
generate_product_images = dbutils.widgets.get("Generate Product Images").lower() == "true"
print(f"{catalog}, {schema}, {image_schema}, {volume_name}, {generate_product_images}")

In [0]:
# spark.sql(f"drop schema if exists {catalog}.{schema} cascade;")
spark.sql(f"create schema if not exists {catalog}.{schema};")

In [0]:
# if generate_product_images:
#   # Generate product images
#   spark.sql(f"drop schema if exists {catalog}.{image_schema} cascade;")
#   spark.sql(f"create schema if not exists {catalog}.{image_schema} managed location 's3://databricks-dkushari/conversational-commerce-search/images';")
  

In [0]:
# # Create Unity Catalog volume for storing AI-generated images
# try:
#     spark.sql(f"""
#         CREATE VOLUME IF NOT EXISTS {catalog}.{image_schema}.product_images
#         COMMENT 'Volume for storing AI-generated product images'
#     """)
#     print("✅ Unity Catalog volume 'product_images' created successfully!")
# except Exception as e:
#     print(f"Volume creation info: {str(e)}")  # May already exist

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, avg, stddev, count
from pyspark.sql.functions import min as spark_min, max as spark_max
from pyspark.sql.types import *
import random as py_random
from faker import Faker
import json
from datetime import datetime, timedelta
import base64
import os

In [0]:
# Initialize Faker for generating realistic data
fake = Faker()
np.random.seed(42)
py_random.seed(42)

## 1. Generate Product Catalog

In [0]:
def generate_products(num_products=10000):
    """Generate synthetic product catalog with rich features"""
    
    # Product categories and their typical attributes
    categories = {
        'Electronics': {
            'subcategories': ['Smartphones', 'Laptops', 'Headphones', 'Tablets', 'Smart Watches', 'Gaming'],
            'brands': ['Apple', 'Samsung', 'Sony', 'Microsoft', 'Google', 'Dell', 'HP', 'Asus'],
            'price_range': (50, 3000),
            'features': ['screen_size', 'battery_life', 'storage', 'ram', 'processor']
        },
        'Clothing': {
            'subcategories': ['Shirts', 'Pants', 'Dresses', 'Shoes', 'Accessories', 'Outerwear'],
            'brands': ['Nike', 'Adidas', 'Zara', 'H&M', 'Uniqlo', 'Levi\'s', 'Gap', 'Forever21'],
            'price_range': (15, 500),
            'features': ['size', 'color', 'material', 'season', 'style']
        },
        'Home & Garden': {
            'subcategories': ['Furniture', 'Kitchen', 'Bathroom', 'Garden', 'Decor', 'Storage'],
            'brands': ['IKEA', 'West Elm', 'Pottery Barn', 'Home Depot', 'Wayfair', 'Target'],
            'price_range': (20, 2000),
            'features': ['material', 'dimensions', 'color', 'style', 'room_type']
        },
        'Books': {
            'subcategories': ['Fiction', 'Non-Fiction', 'Science', 'History', 'Biography', 'Self-Help'],
            'brands': ['Penguin', 'Random House', 'HarperCollins', 'Simon & Schuster', 'Macmillan'],
            'price_range': (10, 50),
            'features': ['genre', 'author', 'publication_year', 'pages', 'format']
        },
        'Sports & Outdoors': {
            'subcategories': ['Fitness', 'Outdoor Gear', 'Sports Equipment', 'Athletic Wear', 'Camping'],
            'brands': ['Nike', 'Adidas', 'Under Armour', 'Patagonia', 'North Face', 'REI'],
            'price_range': (25, 800),
            'features': ['activity_type', 'skill_level', 'season', 'material', 'size']
        }
    }
    
    products = []
    
    for i in range(num_products):
        # Select random category
        category = py_random.choice(list(categories.keys()))
        cat_info = categories[category]
        
        # Generate product details
        subcategory = py_random.choice(cat_info['subcategories'])
        brand = py_random.choice(cat_info['brands'])
        price = round(py_random.uniform(*cat_info['price_range']), 2)
        
        # Generate product name
        product_name = f"{brand} {subcategory} {fake.word().title()}"
        
        # Generate description
        description = fake.text(max_nb_chars=200)
        
        # Generate features specific to category
        features = {}
        for feature in cat_info['features']:
            if feature == 'color':
                features[feature] = fake.color_name()
            elif feature == 'size':
                features[feature] = py_random.choice(['XS', 'S', 'M', 'L', 'XL', 'XXL'])
            elif feature == 'material':
                materials = ['Cotton', 'Polyester', 'Leather', 'Metal', 'Plastic', 'Wood', 'Glass']
                features[feature] = py_random.choice(materials)
            elif 'price' in feature:
                features[feature] = py_random.choice(['Budget', 'Mid-range', 'Premium'])
            else:
                features[feature] = fake.word()
        
        # Generate ratings and reviews
        avg_rating = round(py_random.uniform(2.5, 5.0), 1)
        num_reviews = py_random.randint(0, 1000)
        
        # Generate keywords for text-based features
        keywords = [category.lower(), subcategory.lower(), brand.lower()] + \
                  [fake.word() for _ in range(py_random.randint(3, 8))]
        
        product = {
            'product_id': f'prod_{i:06d}',
            'name': product_name,
            'category': category,
            'subcategory': subcategory,
            'brand': brand,
            'price': price,
            'description': description,
            'avg_rating': avg_rating,
            'num_reviews': num_reviews,
            'features': json.dumps(features),
            'keywords': ','.join(keywords),
            'created_date': fake.date_between(start_date='-2y', end_date='today'),
            'in_stock': py_random.choices([True, False], weights=[0.85, 0.15])[0]
        }
        
        products.append(product)
    
    return products

# Generate products
print("Generating product catalog...")
products_data = generate_products(10000)
print(f"Generated {len(products_data)} products")

# Convert to Spark DataFrame
products_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("subcategory", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("description", StringType(), True),
    StructField("avg_rating", DoubleType(), True),
    StructField("num_reviews", IntegerType(), True),
    StructField("features", StringType(), True),
    StructField("keywords", StringType(), True),
    StructField("created_date", DateType(), True),
    StructField("in_stock", BooleanType(), True)
])

products_df = spark.createDataFrame(products_data, schema=products_schema)
display(products_df)

In [0]:
# # Update the products schema to include AI-generated image fields
# from pyspark.sql import functions as F
# products_schema = StructType([
#     StructField("product_id", StringType(), True),
#     StructField("name", StringType(), True),
#     StructField("category", StringType(), True),
#     StructField("subcategory", StringType(), True),
#     StructField("brand", StringType(), True),
#     StructField("price", DoubleType(), True),
#     StructField("description", StringType(), True),
#     StructField("avg_rating", DoubleType(), True),
#     StructField("num_reviews", IntegerType(), True),
#     StructField("features", StringType(), True),
#     StructField("keywords", StringType(), True),
#     StructField("created_date", DateType(), True),
#     StructField("in_stock", BooleanType(), True),
#     # AI-generated image fields
#     # StructField("image_source", StringType(), True),        # UC Volume path to AI-generated image
# ])

# # print("Updated products schema with AI image fields")

# # Recreate DataFrame with updated schema
# # products_df = spark.createDataFrame(products_data, schema=products_schema)
# # print("Created Spark DataFrame with AI image fields")

# # volume_path = f"/Volumes/{catalog}/{image_schema}/{volume_name}/ai_generated_images/"
# # files_df = (spark.read.format("binaryFile")
# #             .load(volume_path + "*.png")
# #             .select("path"))
# # files_df = files_df.withColumn(
# #     "prod_id",
# #     F.regexp_extract(F.col("path"), r"(prod_\d+)_product_image\.png", 1)
# # ).withColumn(
# #     "path", F.regexp_replace("path", "^dbfs:", "")  # remove dbfs: prefix
# # )
# # products_with_images = (products_df
# #     .join(files_df, products_df["product_id"] == files_df["prod_id"], "left")
# #     .withColumn("image_source", F.col("path"))
# # )
# # category_images = (products_with_images
# #     .filter(F.col("image_source").isNotNull())
# #     .groupBy("category", "subcategory")
# #     .agg(F.first("image_source").alias("fallback_image"))
# # )
# # final_products_df = (products_with_images
# #     .join(category_images, ["category", "subcategory"], "left")
# #     .withColumn(
# #         "image_source",
# #         F.when(F.col("image_source").isNotNull(), F.col("image_source"))
# #          .otherwise(F.col("fallback_image"))
# #     )
# #     .drop("fallback_image").drop("path").drop("prod_id")
# # )

# final_products_df = products_df
# # Show the updated schema
# print("\nUpdated Products Schema:")
# final_products_df.printSchema()


# display(final_products_df.select("product_id", "name", "category", "subcategory"))


## 2. Generate Customer Profiles

In [0]:
def generate_customers(num_customers=100):
    """Generate synthetic customer profiles with preferences"""
    
    customers = []
    
    # Define customer personas with different preferences
    personas = {
        'Tech Enthusiast': {
            'preferred_categories': ['Electronics'],
            'price_sensitivity': 'low',
            'brand_loyalty': 'high',
            'preferred_brands': ['Apple', 'Samsung', 'Sony']
        },
        'Fashion Forward': {
            'preferred_categories': ['Clothing'],
            'price_sensitivity': 'medium',
            'brand_loyalty': 'medium',
            'preferred_brands': ['Zara', 'H&M', 'Nike']
        },
        'Home Decorator': {
            'preferred_categories': ['Home & Garden'],
            'price_sensitivity': 'high',
            'brand_loyalty': 'low',
            'preferred_brands': ['IKEA', 'Target']
        },
        'Book Lover': {
            'preferred_categories': ['Books'],
            'price_sensitivity': 'high',
            'brand_loyalty': 'low',
            'preferred_brands': ['Penguin', 'Random House']
        },
        'Fitness Enthusiast': {
            'preferred_categories': ['Sports & Outdoors'],
            'price_sensitivity': 'medium',
            'brand_loyalty': 'high',
            'preferred_brands': ['Nike', 'Adidas', 'Under Armour']
        },
        'Bargain Hunter': {
            'preferred_categories': ['Electronics', 'Clothing', 'Home & Garden'],
            'price_sensitivity': 'very_high',
            'brand_loyalty': 'very_low',
            'preferred_brands': []
        }
    }
    
    for i in range(num_customers):
        # Assign persona (some customers can be mixed)
        if py_random.random() < 0.8:  # 80% have clear persona
            persona_name = py_random.choice(list(personas.keys()))
            persona = personas[persona_name]
        else:  # 20% are mixed
            persona_name = 'Mixed'
            persona = {
                'preferred_categories': py_random.sample(['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports & Outdoors'], 
                                                    py_random.randint(2, 4)),
                'price_sensitivity': py_random.choice(['low', 'medium', 'high']),
                'brand_loyalty': py_random.choice(['low', 'medium', 'high']),
                'preferred_brands': []
            }
        
        # Generate customer details
        customer = {
            'customer_id': f'cust_{i:03d}',
            'name': fake.name(),
            'email': fake.email(),
            'age': py_random.randint(18, 70),
            'gender': py_random.choice(['M', 'F', 'Other']),
            'location': fake.city(),
            'persona': persona_name,
            'preferred_categories': ','.join(persona['preferred_categories']),
            'price_sensitivity': persona['price_sensitivity'],
            'brand_loyalty': persona['brand_loyalty'],
            'preferred_brands': ','.join(persona['preferred_brands']),
            'registration_date': fake.date_between(start_date='-3y', end_date='-1m'),
            'total_orders': py_random.randint(0, 50),
            'total_spent': round(py_random.uniform(0, 5000), 2)
        }
        
        customers.append(customer)
    
    return customers

# Generate customers
print("Generating customer profiles...")
customers_data = generate_customers(100)
print(f"Generated {len(customers_data)} customers")

# Convert to Spark DataFrame
customers_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("location", StringType(), True),
    StructField("persona", StringType(), True),
    StructField("preferred_categories", StringType(), True),
    StructField("price_sensitivity", StringType(), True),
    StructField("brand_loyalty", StringType(), True),
    StructField("preferred_brands", StringType(), True),
    StructField("registration_date", DateType(), True),
    StructField("total_orders", IntegerType(), True),
    StructField("total_spent", DoubleType(), True)
])

customers_df = spark.createDataFrame(customers_data, schema=customers_schema)
display(customers_df)

## 3. Generate Customer Interactions

In [0]:
def generate_interactions(customers_data, products_data, num_interactions=50000):
    """Generate realistic customer-product interactions"""
    
    interactions = []
    
    # Create lookup dictionaries for faster access
    customers_by_id = {c['customer_id']: c for c in customers_data}
    products_by_category = {}
    for product in products_data:
        category = product['category']
        if category not in products_by_category:
            products_by_category[category] = []
        products_by_category[category].append(product)
    
    interaction_types = ['view', 'cart_add', 'purchase', 'rating', 'review']
    
    for _ in range(num_interactions):
        # Select random customer
        customer = py_random.choice(customers_data)
        customer_id = customer['customer_id']
        
        # Select product based on customer preferences
        preferred_categories = customer['preferred_categories'].split(',')
        
        # 70% chance to interact with preferred category, 30% random
        if py_random.random() < 0.7 and preferred_categories[0] != '':
            category = py_random.choice(preferred_categories)
            if category in products_by_category:
                available_products = products_by_category[category]
            else:
                available_products = products_data
        else:
            available_products = products_data
        
        product = py_random.choice(available_products)
        product_id = product['product_id']
        
        # Generate interaction based on customer behavior
        interaction_type = py_random.choices(
            interaction_types,
            weights=[50, 20, 10, 15, 5],  # views are most common
            k=1
        )[0]
        
        # Generate interaction details
        interaction = {
            'interaction_id': f'int_{len(interactions):08d}',
            'customer_id': customer_id,
            'product_id': product_id,
            'interaction_type': interaction_type,
            'timestamp': fake.date_time_between(start_date='-1y', end_date='now'),
            'session_id': f'sess_{py_random.randint(1000000, 9999999)}'
        }
        
        # Add specific fields based on interaction type
        if interaction_type == 'rating':
            # Bias ratings based on customer preferences and product features
            base_rating = product['avg_rating']
            # Customers tend to rate products they like higher
            if product['category'] in preferred_categories:
                rating = __builtins__.min(5.0, base_rating + py_random.uniform(0, 1))
            else:
                rating = max(1.0, base_rating + py_random.uniform(-1, 0.5))
            interaction['rating'] = round(rating, 1)
            
        elif interaction_type == 'purchase':
            interaction['quantity'] = py_random.randint(1, 3)
            interaction['purchase_amount'] = product['price'] * interaction['quantity']
            
        elif interaction_type == 'view':
            interaction['view_duration'] = py_random.randint(5, 300)  # seconds
            
        interactions.append(interaction)
    
    return interactions

# Generate interactions
print("Generating customer interactions...")
interactions_data = generate_interactions(customers_data, products_data, 50000)
print(f"Generated {len(interactions_data)} interactions")

# Convert to Spark DataFrame
interactions_schema = StructType([
    StructField("interaction_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("interaction_type", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("session_id", StringType(), True),
    StructField("rating", DoubleType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("purchase_amount", DoubleType(), True),
    StructField("view_duration", IntegerType(), True)
])

interactions_df = spark.createDataFrame(interactions_data, schema=interactions_schema)
display(interactions_df)

## 4. Save Data to Delta Tables

In [0]:
# Save products table
prod_table = f"{catalog}.{schema}.products"
spark.sql(f"DROP TABLE IF EXISTS {prod_table};")
products_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(prod_table)

print("Products table saved successfully")

# Save customers table
cust_table = f"{catalog}.{schema}.customers"
spark.sql(f"DROP TABLE IF EXISTS {cust_table};")
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(cust_table)

print("Customers table saved successfully")

# Save interactions table
interactions_table = f"{catalog}.{schema}.interactions"
spark.sql(f"DROP TABLE IF EXISTS {interactions_table};")
interactions_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(interactions_table)

print("Interactions table saved successfully")

In [0]:
query = f"""select product_id, ai_query('databricks-meta-llama-3-3-70b-instruct', "use the product " || name || " of the product with " || product_id || " to generate a description for the product. Just put the description in the response. Do not need to tell what you are doing. Start the descrition with the " || name || " of the product.") new_description from {catalog}.{schema}.products;"""
query

In [0]:
desc_df=spark.sql(query)
display(desc_df)

In [0]:
prod_df=spark.table(f"{catalog}.{schema}.products")
display(prod_df)

In [0]:
from pyspark.sql.functions import coalesce, col

joined_df = prod_df.join(desc_df, on="product_id")

updated_df = joined_df.withColumn("description", col("new_description")) \
                      .drop("new_description")




In [0]:
display(updated_df)

In [0]:
prod_table = f"{catalog}.{schema}.products"
spark.sql(f"DROP TABLE IF EXISTS {prod_table};")
updated_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(prod_table)

## 5. Data Quality Checks and Summary Statistics

In [0]:
print("=== DATA SUMMARY ===")
print(f"Products: {products_df.count():,}")
print(f"Customers: {customers_df.count():,}")
print(f"Interactions: {interactions_df.count():,}")

print("\n=== PRODUCT DISTRIBUTION ===")
display(products_df.groupBy("category").count().orderBy(desc("count")))

print("\n=== CUSTOMER PERSONAS ===")
display(customers_df.groupBy("persona").count().orderBy(desc("count")))

print("\n=== INTERACTION TYPES ===")
display(interactions_df.groupBy("interaction_type").count().orderBy(desc("count")))

print("\n=== PRICE DISTRIBUTION ===")
display(products_df.select(
    spark_min("price").alias("min_price"),
    spark_max("price").alias("max_price"),
    avg("price").alias("avg_price"),
    stddev("price").alias("std_price")
))

print("\n=== RATING DISTRIBUTION ===")
display(interactions_df.filter(col("rating").isNotNull()) \
    .select(
        spark_min("rating").alias("min_rating"),
        spark_max("rating").alias("max_rating"),
        avg("rating").alias("avg_rating"),
        count("rating").alias("total_ratings")
    ))

print("Data generation completed successfully!")

In [ ]:
%sql
-- Materialize per-customer/product interaction features to avoid recomputing them at query time.
-- Run this notebook (or schedule it as a Delta Live Table / workflow) before calling the recommendation UDF.

USE CATALOG retail_consumer_goods;
USE retail_consumer_goods.conversational_commerce_search;
DROP TABLE IF EXISTS retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features;
CREATE TABLE IF NOT EXISTS retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features (
    customer_id STRING,
    product_id STRING,
    interaction_count BIGINT,
    last_interaction_ts TIMESTAMP,
    avg_customer_rating DOUBLE,
    latest_interaction_type STRING,
    interaction_recency_days INT
) USING DELTA
TBLPROPERTIES (
    delta.enableChangeDataFeed = true
);

MERGE INTO retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features AS target
USING (
    WITH base_interactions AS (
        SELECT
            customer_id,
            product_id,
            interaction_type,
            rating,
            timestamp
        FROM retail_consumer_goods.conversational_commerce_search.interactions
    ),
    aggregated AS (
        SELECT
            customer_id,
            product_id,
            COUNT(*) AS interaction_count,
            MAX(timestamp) AS last_interaction_ts,
            AVG(CASE WHEN rating IS NOT NULL THEN rating END) AS avg_customer_rating
        FROM base_interactions
        GROUP BY customer_id, product_id
    ),
    latest AS (
        SELECT DISTINCT
            customer_id,
            product_id,
            FIRST_VALUE(interaction_type) OVER (
                PARTITION BY customer_id, product_id
                ORDER BY timestamp DESC
            ) AS latest_interaction_type
        FROM base_interactions
    )
    SELECT
        a.customer_id,
        a.product_id,
        a.interaction_count,
        a.last_interaction_ts,
        COALESCE(a.avg_customer_rating, 0.0) AS avg_customer_rating,
        l.latest_interaction_type,
        DATEDIFF(CURRENT_DATE(), DATE(a.last_interaction_ts)) AS interaction_recency_days
    FROM aggregated a
    LEFT JOIN latest l
        ON a.customer_id = l.customer_id
       AND a.product_id = l.product_id
) AS source
ON target.customer_id = source.customer_id
AND target.product_id = source.product_id
WHEN MATCHED THEN UPDATE SET
    target.interaction_count = source.interaction_count,
    target.last_interaction_ts = source.last_interaction_ts,
    target.avg_customer_rating = source.avg_customer_rating,
    target.latest_interaction_type = source.latest_interaction_type,
    target.interaction_recency_days = source.interaction_recency_days
WHEN NOT MATCHED THEN INSERT (
    customer_id,
    product_id,
    interaction_count,
    last_interaction_ts,
    avg_customer_rating,
    latest_interaction_type,
    interaction_recency_days
) VALUES (
    source.customer_id,
    source.product_id,
    source.interaction_count,
    source.last_interaction_ts,
    source.avg_customer_rating,
    source.latest_interaction_type,
    source.interaction_recency_days
);



In [ ]:
%sql
-- Synthetic customer-profile feature generation for conversational commerce demos.
-- Prerequisites:
--   * The base tables retail_consumer_goods.conversational_commerce_search.customers,
--     retail_consumer_goods.conversational_commerce_search.products, and
--     retail_consumer_goods.conversational_commerce_search.interactions already exist
--     (they are produced by the 01_data_generation notebook).
--   * Run this script in a Databricks SQL environment connected to the same catalog/schema.

USE CATALOG retail_consumer_goods;
USE retail_consumer_goods.conversational_commerce_search;

-- 1) Generate synthetic browsing telemetry for each customer (mobile + web sessions).
CREATE OR REPLACE TABLE retail_consumer_goods.conversational_commerce_search.customer_browsing_events
USING DELTA
COMMENT 'Synthetic customer browsing sessions across app and web surfaces'
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
)
AS
WITH customer_base AS (
    SELECT
        c.customer_id,
        c.name,
        c.email,
        c.persona,
        c.preferred_categories,
        c.registration_date,
        c.total_orders,
        c.total_spent,
        CASE
            WHEN trim(coalesce(c.preferred_categories, '')) = '' THEN array('General Discovery')
            ELSE transform(split(c.preferred_categories, ','), x -> trim(x))
        END AS pref_categories
    FROM retail_consumer_goods.conversational_commerce_search.customers c
),
expansion AS (
    SELECT
        cb.*,
        GREATEST(4, CAST(FLOOR(rand() * 5) + 4 AS INT)) AS event_target
    FROM customer_base cb
),
event_rows AS (
    SELECT
        e.customer_id,
        e.name,
        e.email,
        e.persona,
        e.pref_categories,
        posexplode(sequence(1, e.event_target)) AS (event_pos, event_index)
    FROM expansion e
),
raw_events AS (
    SELECT
        er.customer_id,
        CONCAT('sess_', substring(md5(CONCAT(er.customer_id, '_', CAST(er.event_index AS STRING))), 1, 16)) AS session_id,
        from_unixtime(unix_timestamp() - CAST(rand() * 86400 * 21 AS BIGINT)) AS event_timestamp,
        CASE WHEN rand() < 0.55 THEN 'mobile_app' ELSE 'website' END AS channel_value,
        CASE
            WHEN rand() < 0.45 THEN 'product_detail'
            WHEN rand() < 0.75 THEN 'search_results'
            WHEN rand() < 0.90 THEN 'category'
            ELSE 'home'
        END AS page_type_value,
        CASE
            WHEN size(er.pref_categories) = 0 THEN 'General Discovery'
            WHEN size(er.pref_categories) = 1 THEN er.pref_categories[0]
            ELSE er.pref_categories[CAST(FLOOR(rand() * size(er.pref_categories)) AS INT)]
        END AS focus_topic_raw,
        CAST(rand() * 280 + 40 AS INT) AS dwell_seconds,
        CAST(rand() * 80 + 15 AS INT) AS scroll_depth_pct
    FROM event_rows er
),
final_events AS (
    SELECT
        customer_id,
        session_id,
        event_timestamp,
        channel_value AS channel,
        page_type_value AS page_type,
        INITCAP(focus_topic_raw) AS focus_topic,
        CASE
            WHEN page_type_value = 'search_results' THEN CONCAT('best ', lower(focus_topic_raw))
            WHEN page_type_value = 'product_detail' THEN CONCAT(lower(focus_topic_raw), ' top picks')
            ELSE CONCAT('latest ', lower(focus_topic_raw))
        END AS search_query,
        dwell_seconds,
        scroll_depth_pct,
        CASE
            WHEN channel_value = 'mobile_app' THEN
                CASE WHEN rand() < 0.6 THEN 'iOS' ELSE 'Android' END
            ELSE
                CASE WHEN rand() < 0.65 THEN 'Desktop' ELSE 'Tablet' END
        END AS device_type,
        CASE WHEN page_type_value = 'product_detail' THEN true ELSE false END AS viewed_product_detail
    FROM raw_events
)
SELECT * FROM final_events;


-- 2) Generate synthetic external signal data (social + engagement cues).
CREATE OR REPLACE TABLE retail_consumer_goods.conversational_commerce_search.customer_external_signals
USING DELTA
COMMENT 'External social-signal telemetry mapped to customers'
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
)
AS
WITH customer_base AS (
    SELECT
        c.customer_id,
        c.name,
        c.email,
        CASE
            WHEN trim(coalesce(c.preferred_categories, '')) = '' THEN array('General Discovery')
            ELSE transform(split(c.preferred_categories, ','), x -> trim(x))
        END AS pref_categories
    FROM retail_consumer_goods.conversational_commerce_search.customers c
),
expansion AS (
    SELECT
        cb.*,
        GREATEST(1, CAST(FLOOR(rand() * 3) + 1 AS INT)) AS signal_target
    FROM customer_base cb
),
signal_rows AS (
    SELECT
        e.customer_id,
        posexplode(sequence(1, e.signal_target)) AS (signal_pos, signal_index),
        e.pref_categories
    FROM expansion e
),
raw_signals AS (
    SELECT
        sr.customer_id,
        from_unixtime(unix_timestamp() - CAST(rand() * 86400 * 30 AS BIGINT)) AS signal_timestamp,
        CASE
            WHEN rand() < 0.40 THEN 'TikTok'
            WHEN rand() < 0.70 THEN 'Instagram'
            WHEN rand() < 0.85 THEN 'YouTube'
            WHEN rand() < 0.95 THEN 'Pinterest'
            ELSE 'Reddit'
        END AS platform,
        CASE
            WHEN rand() < 0.35 THEN 'trend_follow'
            WHEN rand() < 0.60 THEN 'content_engagement'
            WHEN rand() < 0.80 THEN 'influencer_like'
            WHEN rand() < 0.95 THEN 'wishlist_save'
            ELSE 'cart_discussion'
        END AS signal_type,
        CASE
            WHEN size(sr.pref_categories) = 0 THEN 'general lifestyle'
            WHEN size(sr.pref_categories) = 1 THEN sr.pref_categories[0]
            ELSE sr.pref_categories[CAST(FLOOR(rand() * size(sr.pref_categories)) AS INT)]
        END AS content_topic_raw,
        ROUND(rand() * 0.7 + 0.3, 3) AS engagement_score
    FROM signal_rows sr
)
SELECT
    customer_id,
    signal_timestamp,
    platform,
    signal_type,
    INITCAP(content_topic_raw) AS content_topic,
    engagement_score,
    CONCAT(
        'https://www.',
        lower(regexp_replace(platform, '\\s+', '')),
        '.com/',
        lower(regexp_replace(content_topic_raw, '\\s+', '-')),
        '/',
        substring(md5(concat(platform, content_topic_raw, CAST(signal_timestamp AS STRING))), 1, 8)
    ) AS source_url
FROM raw_signals;


-- 3) Basket intelligence (co-purchase graph per customer) sourced from interaction history.
CREATE OR REPLACE TABLE retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence
USING DELTA
COMMENT 'Customer-level product affinity pairs derived from purchase baskets'
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
)
AS
WITH purchase_events AS (
    SELECT
        customer_id,
        session_id,
        collect_list(product_id) AS products,
        MAX(timestamp) AS last_purchase_ts
    FROM retail_consumer_goods.conversational_commerce_search.interactions
    WHERE lower(interaction_type) = 'purchase'
    GROUP BY customer_id, session_id
    HAVING size(products) > 1
),
pair_rows AS (
    SELECT
        pe.customer_id,
        pe.last_purchase_ts,
        anchor_pos,
        anchor_product_id,
        assoc_pos,
        associated_product_id
    FROM purchase_events pe
    LATERAL VIEW posexplode(pe.products) anchor AS anchor_pos, anchor_product_id
    LATERAL VIEW posexplode(pe.products) assoc AS assoc_pos, associated_product_id
    WHERE anchor_pos <> assoc_pos
),
pair_scores AS (
    SELECT
        pr.customer_id,
        pr.anchor_product_id,
        pr.associated_product_id,
        COUNT(*) AS pair_count,
        MAX(pr.last_purchase_ts) AS last_seen_purchase_ts
    FROM pair_rows pr
    GROUP BY pr.customer_id, pr.anchor_product_id, pr.associated_product_id
),
normalized AS (
    SELECT
        ps.*,
        ps.pair_count / SUM(ps.pair_count) OVER (PARTITION BY ps.customer_id, ps.anchor_product_id) AS affinity_share,
        ROW_NUMBER() OVER (
            PARTITION BY ps.customer_id, ps.anchor_product_id
            ORDER BY ps.pair_count DESC, ps.associated_product_id
        ) AS affinity_rank
    FROM pair_scores ps
)
SELECT
    n.customer_id,
    c.name AS customer_name,
    n.anchor_product_id,
    anchor.name AS anchor_product_name,
    n.associated_product_id,
    assoc.name AS associated_product_name,
    n.pair_count,
    ROUND(n.affinity_share, 4) AS affinity_share,
    n.last_seen_purchase_ts,
    n.affinity_rank
FROM normalized n
LEFT JOIN retail_consumer_goods.conversational_commerce_search.products anchor
    ON n.anchor_product_id = anchor.product_id
LEFT JOIN retail_consumer_goods.conversational_commerce_search.products assoc
    ON n.associated_product_id = assoc.product_id
LEFT JOIN retail_consumer_goods.conversational_commerce_search.customers c
    ON n.customer_id = c.customer_id
WHERE n.affinity_rank <= 5;


-- 4) Unified customer profile snapshot that surfaces recent purchases, browsing, social signals, and basket insights.
CREATE OR REPLACE TABLE retail_consumer_goods.conversational_commerce_search.customer_profile_summary
USING DELTA
COMMENT 'Admin-facing customer profile rollup combining transactions, browsing, social signals, and basket intelligence'
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
)
AS
WITH base_customers AS (
    SELECT
        customer_id,
        name,
        email,
        persona,
        preferred_categories,
        registration_date,
        total_orders,
        total_spent
    FROM retail_consumer_goods.conversational_commerce_search.customers
),
recent_purchase AS (
    SELECT
        customer_id,
        product_id,
        purchase_amount,
        timestamp AS purchase_ts,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY timestamp DESC) AS rn
    FROM retail_consumer_goods.conversational_commerce_search.interactions
    WHERE lower(interaction_type) = 'purchase'
),
recent_purchase_enriched AS (
    SELECT
        rp.customer_id,
        rp.product_id,
        p.name AS product_name,
        p.category AS product_category,
        p.price AS listed_price,
        rp.purchase_amount,
        rp.purchase_ts AS last_purchase_ts
    FROM recent_purchase rp
    LEFT JOIN retail_consumer_goods.conversational_commerce_search.products p
        ON rp.product_id = p.product_id
    WHERE rp.rn = 1
),
recent_browse AS (
    SELECT
        customer_id,
        session_id,
        event_timestamp,
        channel,
        page_type,
        device_type,
        search_query,
        focus_topic,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY event_timestamp DESC) AS rn
    FROM retail_consumer_goods.conversational_commerce_search.customer_browsing_events
),
recent_external AS (
    SELECT
        customer_id,
        signal_timestamp,
        platform,
        signal_type,
        content_topic,
        engagement_score,
        source_url,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY signal_timestamp DESC) AS rn
    FROM retail_consumer_goods.conversational_commerce_search.customer_external_signals
),
basket_ranked AS (
    SELECT
        cbi.*,
        ROW_NUMBER() OVER (
            PARTITION BY cbi.customer_id
            ORDER BY cbi.affinity_share DESC, cbi.pair_count DESC
        ) AS rn
    FROM retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence cbi
),
basket_summary AS (
    SELECT
        customer_id,
        array_join(
            transform(
                array_sort(
                    collect_list(
                        named_struct(
                            'rn', rn,
                            'text',
                            CONCAT(
                                coalesce(associated_product_name, associated_product_id),
                                ' (',
                                ROUND(affinity_share * 100, 1),
                                '%)'
                            )
                        )
                    )
                ),
                x -> x.text
            ),
            '; '
        ) AS top_cross_sell_products
    FROM basket_ranked
    WHERE rn <= 3
    GROUP BY customer_id
)
SELECT
    bc.customer_id,
    bc.name AS customer_name,
    bc.email,
    bc.persona,
    bc.preferred_categories,
    bc.registration_date,
    bc.total_orders,
    bc.total_spent,
    rpe.product_id AS last_product_id,
    rpe.product_name AS last_product_name,
    rpe.product_category AS last_product_category,
    rpe.last_purchase_ts,
    rpe.purchase_amount,
    rb.event_timestamp AS last_browse_ts,
    rb.channel AS last_browse_channel,
    rb.device_type AS last_browse_device,
    rb.page_type AS last_browse_page,
    rb.search_query AS last_browse_search,
    rb.focus_topic AS last_browse_topic,
    re.signal_timestamp AS last_external_ts,
    re.platform AS last_external_platform,
    re.signal_type AS last_external_signal_type,
    re.content_topic AS last_external_topic,
    re.engagement_score AS last_external_engagement_score,
    re.source_url AS last_external_url,
    bs.top_cross_sell_products
FROM base_customers bc
LEFT JOIN recent_purchase_enriched rpe
    ON bc.customer_id = rpe.customer_id
LEFT JOIN recent_browse rb
    ON bc.customer_id = rb.customer_id AND rb.rn = 1
LEFT JOIN recent_external re
    ON bc.customer_id = re.customer_id AND re.rn = 1
LEFT JOIN basket_summary bs
    ON bc.customer_id = bs.customer_id;



In [ ]:
-- Generate Synthetic Basket Intelligence Data
-- This creates frequently-bought-together product pairs for each customer
-- based on their purchase history and preferences

USE CATALOG retail_consumer_goods;
USE SCHEMA conversational_commerce_search;

-- Drop and recreate the basket intelligence table with synthetic data
CREATE OR REPLACE TABLE retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence
USING DELTA
COMMENT 'Customer-level product affinity pairs derived from purchase baskets (synthetic)'
TBLPROPERTIES (
  delta.enableChangeDataFeed = true
)
AS
WITH customer_purchases AS (
    -- Get all customers and their purchased products
    SELECT DISTINCT
        i.customer_id,
        c.name AS customer_name,
        i.product_id,
        p.name AS product_name,
        p.category,
        p.subcategory
    FROM retail_consumer_goods.conversational_commerce_search.interactions i
    JOIN retail_consumer_goods.conversational_commerce_search.products p 
        ON i.product_id = p.product_id
    JOIN retail_consumer_goods.conversational_commerce_search.customers c
        ON i.customer_id = c.customer_id
    WHERE LOWER(i.interaction_type) = 'purchase'
),
customer_product_pairs AS (
    -- Create product pairs for each customer (cartesian product of their purchases)
    SELECT
        cp1.customer_id,
        cp1.customer_name,
        cp1.product_id AS anchor_product_id,
        cp1.product_name AS anchor_product_name,
        cp2.product_id AS associated_product_id,
        cp2.product_name AS associated_product_name,
        -- Products from same category are more likely to be bought together
        CASE 
            WHEN cp1.category = cp2.category THEN 0.7
            ELSE 0.3
        END AS base_affinity
    FROM customer_purchases cp1
    JOIN customer_purchases cp2
        ON cp1.customer_id = cp2.customer_id
        AND cp1.product_id < cp2.product_id  -- Avoid duplicates (A,B) and (B,A)
),
synthetic_pairs AS (
    -- Generate synthetic purchase counts and affinity scores
    SELECT
        customer_id,
        customer_name,
        anchor_product_id,
        anchor_product_name,
        associated_product_id,
        associated_product_name,
        -- Random pair count between 1 and 5
        CAST(FLOOR(rand() * 5) + 1 AS INT) AS pair_count,
        -- Affinity score based on base affinity + some randomness
        ROUND(base_affinity * (0.5 + rand() * 0.5), 4) AS affinity_share,
        CURRENT_TIMESTAMP() AS last_seen_purchase_ts
    FROM customer_product_pairs
    -- Add some randomness - not all product pairs should appear
    WHERE rand() > 0.7  -- Only keep 30% of possible pairs to make it realistic
),
ranked_pairs AS (
    -- Rank pairs by affinity for each customer and anchor product
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY customer_id, anchor_product_id
            ORDER BY affinity_share DESC, associated_product_id
        ) AS affinity_rank
    FROM synthetic_pairs
)
-- Keep top 5 associated products for each anchor product per customer
SELECT
    customer_id,
    customer_name,
    anchor_product_id,
    anchor_product_name,
    associated_product_id,
    associated_product_name,
    pair_count,
    affinity_share,
    last_seen_purchase_ts,
    affinity_rank
FROM ranked_pairs
WHERE affinity_rank <= 5
ORDER BY customer_id, anchor_product_id, affinity_rank;

-- Verify the data
SELECT 
    COUNT(DISTINCT customer_id) AS num_customers_with_baskets,
    COUNT(*) AS total_basket_pairs,
    ROUND(AVG(pair_count), 2) AS avg_pair_count,
    ROUND(AVG(affinity_share), 4) AS avg_affinity_share
FROM retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence;

-- Show sample data for first customer
SELECT *
FROM retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence
LIMIT 10;



In [ ]:
%sql
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customer_basket_intelligence CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customer_browsing_events CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customer_external_signals CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customer_profile_summary CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.customers CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.interactions CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.products CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.product_features CLUSTER BY AUTO;
ALTER TABLE retail_consumer_goods.conversational_commerce_search.user_features CLUSTER BY AUTO;

In [0]:
# def get_ai_product_image(product_id, category, subcategory, brand, name, description):
#     """
#     Generate AI product images using Databricks Shutterstock ImageAI model
#     and store them in Unity Catalog volume
#     """

#     image_filename = f"{product_id}_product_image.png"
#     image_path = f"/Volumes/{catalog}/{image_schema}/{volume_name}/ai_generated_images/{image_filename}"
    
#     if generate_product_images:
#       # Create detailed prompt for AI image generation
#       prompt = f"""A high-quality product photograph of {name}, a {subcategory} from {brand} in the {category} category. Professional product photography, clean white background, well-lit, commercial quality."""
      
#       try:
#           query = f"""SELECT ai_query('databricks-shutterstock-imageai', 
#                               request => '{prompt}');"""
#           response= spark.sql(query)
          
#           base64_string = response.collect()[0]
#           image_data = base64.b64decode(base64_string[0])
          
#           print(f"Saving image to {image_path}")
#           with open(image_path, "wb") as f:
#               f.write(image_data)
#       except Exception as e:
#         print(f"Error generating image for {product_id}: {str(e)}")
#         return {
#             'image_source': None,
#             'generation_status': 'failed',
#             'prompt_used': prompt,
#             'error': str(e)
#         }     
#     return {
#         'image_source': image_path
#     }

# print("✅ AI image generation function defined!")


In [0]:

# # Generate AI images for products (sample batch for demonstration)
# print("Getting AI images for products...")

# updated_products = []

# for i, product in enumerate(products_data):
#     print(f"Generating AI image for product {i+1}: {product['name']}")
    
#     image_data = get_ai_product_image(
#         product['product_id'], 
#         product['category'], 
#         product['subcategory'], 
#         product['brand'], 
#         product['name'],
#         product['description']
#     )
    
#     # Add image fields to product
#     product.update(image_data)
#     updated_products.append(product)

# # Replace the original products_data
# products_data = updated_products
# print(f"✅ Generated AI images for {len(products_data)} products")

# # Show sample with AI-generated images
# print("\n🤖 Sample products with AI-generated images:")
# for i, product in enumerate(products_data[:5]):
#     print(f"\n📦 Product {i+1}: {product['name']}")
#     print(f"   📂 Category: {product['category']} > {product['subcategory']}")
#     print(f"   🏷️  Brand: {product['brand']}")
#     print(f"   🎯 Generation Status: {product['generation_status']}")
#     if product['image_source']:
#         print(f"   🖼️  Image Path: {product['image_source']}")
#     if product['prompt_used']:
#         print(f"   💭 AI Prompt: {product['prompt_used'][:100]}...")

